In [10]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [11]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [12]:
import sys 
sys.path.insert(0,'drive/colab_kullanım')

In [13]:
!pip install -q keras

In [14]:
!curl http://www.gumuskaya.com/gtsrb.zip >> gtsrb.zip
!unzip gtsrb.zip

Görüntülenen çıkış son 5000 satıra kısaltıldı.
  inflating: gtsrb/2/00012_00019.ppm  
  inflating: gtsrb/2/00049_00024.ppm  
  inflating: gtsrb/2/00021_00023.ppm  
  inflating: gtsrb/2/00030_00004.ppm  
  inflating: gtsrb/2/00030_00010.ppm  
  inflating: gtsrb/2/00049_00018.ppm  
  inflating: gtsrb/2/00030_00006.ppm  
  inflating: gtsrb/2/00021_00021.ppm  
  inflating: gtsrb/2/00030_00012.ppm  
  inflating: gtsrb/2/00021_00009.ppm  
  inflating: gtsrb/2/00049_00026.ppm  
  inflating: gtsrb/2/00003_00028.ppm  
  inflating: gtsrb/2/00044_00025.ppm  
  inflating: gtsrb/2/00003_00014.ppm  
  inflating: gtsrb/2/00012_00027.ppm  
  inflating: gtsrb/2/00003_00000.ppm  
  inflating: gtsrb/2/00044_00019.ppm  
  inflating: gtsrb/2/00022_00015.ppm  
  inflating: gtsrb/2/00022_00001.ppm  
  inflating: gtsrb/2/00033_00026.ppm  
  inflating: gtsrb/2/00022_00029.ppm  
  inflating: gtsrb/2/00047_00005.ppm  
  inflating: gtsrb/2/00047_00011.ppm  
  inflating: gtsrb/2/00000_00008.ppm  
  inflating: gtsr

In [42]:
import cv2
import numpy as np
import os
import sys
import tensorflow as tf

from sklearn.model_selection import train_test_split

EPOCHS = 10
IMG_WIDTH = 30
IMG_HEIGHT = 30
NUM_CATEGORIES = 43
TEST_SIZE = 0.4


def main():
    
    # Check command-line arguments
    if len(sys.argv) not in [2, 3]:
        sys.exit("Usage: python traffic.py data_directory [model.h5]")

    # Get image arrays and labels for all image files
    images, labels = load_data(sys.argv[1])

    # Split data into training and testing sets
    labels = tf.keras.utils.to_categorical(labels)
    x_train, x_test, y_train, y_test = train_test_split(
        np.array(images), np.array(labels), test_size=TEST_SIZE
    )

    # Get a compiled neural network
    model = get_model()

    # Fit model on training data
    model.fit(x_train, y_train, epochs=EPOCHS)

    # Evaluate neural network performance
    model.evaluate(x_test,  y_test, verbose=2)

    # Save model to file
    if len(sys.argv) == 3:
        filename = sys.argv[2]
        model.save(filename)
        print(f"Model saved to {filename}.")


def load_data(data_dir):
    """
    Load image data from directory `data_dir`.

    Assume `data_dir` has one directory named after each category, numbered
    0 through NUM_CATEGORIES - 1. Inside each category directory will be some
    number of image files.

    Return tuple `(images, labels)`. `images` should be a list of all
    of the images in the data directory, where each image is formatted as a
    numpy ndarray with dimensions IMG_WIDTH x IMG_HEIGHT x 3. `labels` should
    be a list of integer labels, representing the categories for each of the
    corresponding `images`.
    """
    
    list_of_image = []
    list_of_label = []

    image_files = os.walk("gtsrb")

    
    for r, d, f in image_files:
        for file in f:
            if not file.endswith(".ppm"):
                continue
            images = cv2.imread(os.path.join(r, file), True)
            images = cv2.resize(images, dsize=(30, 30))

            #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Augment an image
            #transformed = transform(image=image)
            #transformed_image = transformed["image"]

            list_of_image.append(images/255.)
            list_of_label.append(os.path.basename(r))
    
    return list_of_image, list_of_label


def get_model():
    """
    Returns a compiled convolutional neural network model. Assume that the
    `input_shape` of the first layer is `(IMG_WIDTH, IMG_HEIGHT, 3)`.
    The output layer should have `NUM_CATEGORIES` units, one for each category.
    """

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=(30, 30, 3)))
    model.add(tf.keras.layers.Conv2D(512, (3,3)))
    model.add(tf.keras.layers.Activation(tf.keras.activations.relu))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Conv2D(256, (3,3)))
    model.add(tf.keras.layers.Activation(tf.keras.activations.relu))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Conv2D(256, (3,3)))
    model.add(tf.keras.layers.Activation(tf.keras.activations.relu))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Conv2D(128, (3,3)))
    model.add(tf.keras.layers.Activation(tf.keras.activations.relu))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Conv2D(128, (3,3)))
    model.add(tf.keras.layers.Activation(tf.keras.activations.relu))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Conv2D(128, (3,3)))
    model.add(tf.keras.layers.Activation(tf.keras.activations.relu))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(64))
    model.add(tf.keras.layers.Activation(tf.keras.activations.relu))
    model.add(tf.keras.layers.Dense(43))
    model.add(tf.keras.layers.Activation(tf.keras.activations.softmax))
    model.compile(loss="mse", optimizer="Adam", metrics=["acc"])
    

    return model


In [43]:
main()

Epoch 1/10
272/500 [===============>..............] - ETA: 8s - loss: 0.0226 - acc: 0.0519

KeyboardInterrupt: ignored